In [2]:
import json
import pandas as pd
import math
from numpy import diff
from numpy import gradient
import numpy as np
from scipy import pi
from scipy import fft
from scipy.fft import fft, fftfreq
from scipy import signal
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

def lowpass_filter(column,dict):#column, cutoff, order, sample_rate):
    cutoff = dict['cutoff']
    order = dict['order']  
    rate = dict['rate']
    b,a = signal.butter(order,cutoff/(rate/2),btype='lowpass')
    filtered_channel = signal.filtfilt(b,a,column)
    return (filtered_channel)

def get_psd(df,fs,bin_width):
    
    #fs = len(df)/(df.index[-1]-df.index[0])    
    f, psd = signal.welch(df.to_numpy(), 
                          fs=fs, 
                          nperseg=fs/bin_width,
                          #window='hanning',
                          axis=0
                         )

    df_psd = pd.DataFrame(psd)#,columns=df.columns)
    df_psd.columns
    df_psd['Frequency (Hz)'] = f
    df_psd = df_psd.set_index('Frequency (Hz)')
    
    return df_psd[1:] #drop the first value because it makes the plots look bad and is effectively 0

# gps_data = data['locations']  # extra GPS entry
# frame=pd.DataFrame(gps_data) # Convert to data frame

# frame['elapsed_time'] = frame['tUTC'] - frame['tUTC'][0] # change time values to relative for this run

# susp=data['suspension']
# df=pd.DataFrame(susp)  # choose suspension entry and create dataframe
# df=df[:snip]


#clip zero values/sensor dropouts & generate velocity columns
def process_sensor_data(df,rate):
    size=len(df.index)
    timedata,interval=np.linspace(0,size/rate,num=size,retstep=True, endpoint=False) #create timestamps
    df['time']=np.round(timedata,3) # add to dataframe, round to 0.001s
    df.where(df.fork>0,inplace=True)
    df.where(df.shock>0,inplace=True)
    df.where(df.rearWheel>0,inplace=True)
    x=df.min()
    df.fork=df.fork-x['fork']
    df.shock=df.shock-x['shock']
    df.rearWheel=df.rearWheel-x['rearWheel']
    df['forkVelocity']=np.gradient(df.fork,df.time)
    df['shockVelocity']=np.gradient(df.shock,df.time)
    df['rearVelocity']=np.gradient(df.rearWheel,df.time)
    return df

# rider_data = {
#     'forktravel' : 160,
#     'wheeltravel' : 146,
#     'shocktravel' : 60,
# }    

In [3]:

df=pd.read_csv('mtb_phd/minerals/OPPOFindX5Pro-03-15-02/OPPOFindX5Pro-03-15-02.csv',usecols=[1,2,3,4,5], 
               names=( 'fork','rearWheel','frontForce','shock','rearForce'), header=0)
df

,fork,rearWheel,frontForce,shock,rearForce
0,44.56,38.01,1.42,14.03,0.78
1,44.61,38.01,1.42,14.03,0.78
2,45.24,37.96,-0.85,14.01,1.80
3,46.42,37.96,-0.85,14.01,1.80
4,48.08,37.91,-1.00,13.99,0.99
...,...,...,...,...,...
47544,30.94,-9.99,-1.16,-9.99,0.00
47545,30.94,-9.99,-1.16,-9.99,0.00
47546,30.99,-9.99,-1.14,-9.99,0.00
47547,30.99,-9.99,-1.14,-9.99,0.00


In [4]:
df2=process_sensor_data(df,200)
df2

,fork,rearWheel,frontForce,shock,rearForce,time,forkVelocity,shockVelocity,rearVelocity
0,24.46,32.80,1.42,12.21,0.78,0.000,10.0,0.0,0.0
1,24.51,32.80,1.42,12.21,0.78,0.005,68.0,-2.0,-5.0
2,25.14,32.75,-0.85,12.19,1.80,0.010,181.0,-2.0,-5.0
3,26.32,32.75,-0.85,12.19,1.80,0.015,284.0,-2.0,-5.0
4,27.98,32.70,-1.00,12.17,0.99,0.020,322.0,-4.0,-9.0
...,...,...,...,...,...,...,...,...,...
47544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df2.dropna(how='any', inplace=True)

In [13]:
df2.tail()

,fork,rearWheel,frontForce,shock,rearForce,time,forkVelocity,shockVelocity,rearVelocity
47538,10.84,3.82,-1.14,1.36,0.53,237.690,-5.0,2.000000e+00,5.000000e+00
47539,10.79,3.82,-1.14,1.36,0.53,237.695,-5.0,-2.000000e+00,-5.000000e+00
47540,10.79,3.77,-1.17,1.34,0.52,237.700,10.0,-2.273737e-11,-5.684342e-11
47541,10.89,3.82,-1.17,1.36,0.52,237.705,5.0,4.000000e+00,1.000000e+01
47542,10.84,3.87,-1.17,1.38,0.51,237.710,-5.0,2.000000e+00,5.000000e+00


In [15]:
import plotly.figure_factory as ff
a=df2['forkVelocity']
b=df2['rearVelocity']
hist_data = [a,b]
group_labels = ['fork','rear wheel'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels,show_rug=False, bin_size=25)
# fig.update_yaxes(range=[0,0.05])
# fig.update_xaxes(range=[1,140])
fig.show()